In [ ]:
-- Start a SQL Agent Job
CREATE PROCEDURE [dbo].[spStartAgentJob]
	   @Jobname VARCHAR(180)
		WITH EXECUTE AS 'Domain\Username'  -- <- Must Change
AS
BEGIN

BEGIN TRY
Exec('msdb.dbo.sp_start_job  @job_name = '+ @Jobname)
Select  'Running Job: ' + @Jobname as [Status]
END TRY
BEGIN CATCH
Select  ERROR_MESSAGE()  as [Status]
END CATCH
END

In [ ]:
-- Run a file SSIS Package

EXEC sp_configure 'show advanced options', 1
GO
-- To update the currently configured value for advanced options.
RECONFIGURE
GO
-- To enable the feature.
EXEC sp_configure 'xp_cmdshell', 1
GO
-- To update the currently configured value for this feature.
RECONFIGURE
GO　

CREATE PROCEDURE [dbo].[sp_RunSSIS64]
	   @pFile NVARCHAR(400),
       @pPass NVARCHAR(25)
AS
BEGIN
SET @pFile  = 'call "C:\Program Files\Microsoft SQL Server\150\DTS\Binn\dtexec.exe" /file "' + @pFile + '" /de ' + @pPass 
EXEC master..xp_cmdshell @pFile
END

/*
-- DISABLE XP_CMDSHELL - 
-- Disallow further advanced options to be changed.
EXEC sp_configure 'show advanced options', 0
GO
-- Update the currently configured value for advanced options.
RECONFIGURE
GO
*/

In [ ]:
-- Run a Powershell file
EXEC sp_configure 'show advanced options', 1
GO
-- To update the currently configured value for advanced options.
RECONFIGURE
GO
-- To enable the feature.
EXEC sp_configure 'xp_cmdshell', 1
GO
-- To update the currently configured value for this feature.
RECONFIGURE
GO　

--create stored procedure　
CREATE PROCEDURE spPowershell
 @pFile NVARCHAR(500)
AS
BEGIN
SET @pFile  = 'powershell.exe -ExecutionPolicy Bypass -File "' + @pFile + '" '
EXEC master..xp_cmdshell @pFile
END

/*
-- DISABLE XP_CMDSHELL - 
-- Disallow further advanced options to be changed.
EXEC sp_configure 'show advanced options', 0
GO
-- Update the currently configured value for advanced options.
RECONFIGURE
GO
*/


In [ ]:
-- To search all tables and columns for a given search string
CREATE PROCEDURE [dbo].[spSearchAllTables]
(
	@SearchStr nvarchar(100)
)
AS
BEGIN
	-- Copyright © 2002 Narayana Vyas Kondreddi. All rights reserved.

	CREATE TABLE #Results (ColumnName nvarchar(370), ColumnValue nvarchar(3630))

	SET NOCOUNT ON

	DECLARE @TableName nvarchar(256), @ColumnName nvarchar(128), @SearchStr2 nvarchar(110)
	SET  @TableName = ''
	SET @SearchStr2 = QUOTENAME('%' + @SearchStr + '%','''')

	WHILE @TableName IS NOT NULL
	BEGIN
		SET @ColumnName = ''
		SET @TableName = 
		(
			SELECT MIN(QUOTENAME(TABLE_SCHEMA) + '.' + QUOTENAME(TABLE_NAME))
			FROM 	INFORMATION_SCHEMA.TABLES
			WHERE 		TABLE_TYPE = 'BASE TABLE'
				AND	QUOTENAME(TABLE_SCHEMA) + '.' + QUOTENAME(TABLE_NAME) > @TableName
				AND	OBJECTPROPERTY(
						OBJECT_ID(
							QUOTENAME(TABLE_SCHEMA) + '.' + QUOTENAME(TABLE_NAME)
							 ), 'IsMSShipped'
						       ) = 0
		)

		WHILE (@TableName IS NOT NULL) AND (@ColumnName IS NOT NULL)
		BEGIN
			SET @ColumnName =
			(
				SELECT MIN(QUOTENAME(COLUMN_NAME))
				FROM 	INFORMATION_SCHEMA.COLUMNS
				WHERE 		TABLE_SCHEMA	= PARSENAME(@TableName, 2)
					AND	TABLE_NAME	= PARSENAME(@TableName, 1)
					AND	DATA_TYPE IN ('char', 'varchar', 'nchar', 'nvarchar')
					AND	QUOTENAME(COLUMN_NAME) > @ColumnName
			)
	
			IF @ColumnName IS NOT NULL
			BEGIN
				INSERT INTO #Results
				EXEC
				(
					'SELECT ''' + @TableName + '.' + @ColumnName + ''', LEFT(' + @ColumnName + ', 3630) 
					FROM ' + @TableName + ' (NOLOCK) ' +
					' WHERE ' + @ColumnName + ' LIKE ' + @SearchStr2
				)
			END
		END	
	END

	SELECT ColumnName, ColumnValue FROM #Results
END

In [ ]:
-- To search all columns in a table for a given search string
CREATE PROCEDURE [dbo].[spSearchAllInTable]
(
	@SearchStr nvarchar(100),
	@TableName nvarchar(256)
)
AS
BEGIN

	-- Org Copyright © 2002 Narayana Vyas Kondreddi. All rights reserved.

	CREATE TABLE #Results (ColumnName nvarchar(370), ColumnValue nvarchar(3630))

	SET NOCOUNT ON

	DECLARE @ColumnName nvarchar(128), @SearchStr2 nvarchar(110)
	SET @SearchStr2 = QUOTENAME('%' + @SearchStr + '%','''')

	BEGIN
		SET @ColumnName = ''

		WHILE (@TableName IS NOT NULL) AND (@ColumnName IS NOT NULL)
		BEGIN
			SET @ColumnName =
			(
				SELECT MIN(QUOTENAME(COLUMN_NAME))
				FROM 	INFORMATION_SCHEMA.COLUMNS
				WHERE 		TABLE_SCHEMA	= PARSENAME(@TableName, 2)
					AND	TABLE_NAME	= PARSENAME(@TableName, 1)
					AND	DATA_TYPE IN ('char', 'varchar', 'nchar', 'nvarchar')
					AND	QUOTENAME(COLUMN_NAME) > @ColumnName
			)
	
			IF @ColumnName IS NOT NULL
			BEGIN
				INSERT INTO #Results
				EXEC
				(
					'SELECT ''' + @TableName + '.' + @ColumnName + ''', LEFT(' + @ColumnName + ', 3630) 
					FROM ' + @TableName + ' (NOLOCK) ' +
					' WHERE ' + @ColumnName + ' LIKE ' + @SearchStr2
				)
			END
		END	
END

	SELECT ColumnName, ColumnValue FROM #Results
END

In [ ]:
-- Issue GRANT EXEC statement for all stored procedures for a user
CREATE PROCEDURE [dbo].[spGrantExecAllStoredProc] @user sysname AS
SET NOCOUNT ON

-- 1 - Variable declarations
DECLARE @CMD1 varchar(8000)
DECLARE @MAXOID int
DECLARE @OwnerName varchar(128)
DECLARE @ObjectName varchar(128)

-- 2 - Create temporary table
CREATE TABLE #StoredProcedures
(OID int IDENTITY (1,1),
StoredProcOwner varchar(128) NOT NULL,
StoredProcName varchar(128) NOT NULL)

-- 3 - Populate temporary table
INSERT INTO #StoredProcedures (StoredProcOwner, StoredProcName)
SELECT ROUTINE_SCHEMA, ROUTINE_NAME
FROM INFORMATION_SCHEMA.ROUTINES
WHERE ROUTINE_NAME NOT LIKE 'dt_%'
AND ROUTINE_TYPE = 'PROCEDURE'

-- 4 - Capture the @MAXOID value
SELECT @MAXOID = MAX(OID) FROM #StoredProcedures

-- 5 - WHILE loop
WHILE @MAXOID > 0
BEGIN

-- 6 - Initialize the variables
SELECT @OwnerName = StoredProcOwner,
@ObjectName = StoredProcName
FROM #StoredProcedures
WHERE OID = @MAXOID

-- 7 - Build the string
SELECT @CMD1 = 'GRANT EXEC ON ' + '[' + @OwnerName + ']' + '.' + '[' + @ObjectName + ']' + ' TO ' + @user
--SELECT @CMD1 = 'GRANT EXEC ON [' + @OwnerName + '].[' + @ObjectName + '] TO [' + @user + ']'

-- 8 - Execute the string
-- SELECT @CMD1
EXEC(@CMD1)

-- 9 - Decrement @MAXOID
SET @MAXOID = @MAXOID - 1
END

-- 10 - Drop the temporary table
DROP TABLE #StoredProcedures

SET NOCOUNT OFF

In [ ]:
-- Check SQL Statement is vaild
CREATE	PROCEDURE [dbo].[spValidateSqlSyntax](
@CustomQuery AS NVARCHAR(MAX),
@StatusMsg AS VARCHAR(MAX) OUTPUT
)
AS
BEGIN
BEGIN TRY

IF @CustomQuery IS NOT NULL BEGIN 
SET @CustomQuery =  'SET NOEXEC ON; ' + @CustomQuery + ' ; SET NOEXEC OFF;'

END

EXEC sp_executesql @CustomQuery
SET @StatusMsg = 'Success'

END TRY
BEGIN CATCH
SET @StatusMsg = ERROR_MESSAGE() 
END CATCH
END

In [ ]:
-- Write data to text file to a file destination. 
CREATE PROCEDURE spWriteStringToFile
 (
@String Varchar(max), 
@Path VARCHAR(255),
@Filename VARCHAR(100)

)
AS
DECLARE  @objFileSystem int
        ,@objTextStream int,
        @objErrorObject int,
        @strErrorMessage Varchar(1000),
        @Command varchar(1000),
        @hr int,
        @fileAndPath varchar(80)

set nocount on

select @strErrorMessage='opening the File System Object'
EXECUTE @hr = sp_OACreate  'Scripting.FileSystemObject' , @objFileSystem OUT

Select @FileAndPath=@path+'\'+@filename
if @HR=0 Select @objErrorObject=@objFileSystem , @strErrorMessage='Creating file "'+@FileAndPath+'"'
if @HR=0 execute @hr = sp_OAMethod   @objFileSystem   , 'CreateTextFile'
    , @objTextStream OUT, @FileAndPath,2,True

if @HR=0 Select @objErrorObject=@objTextStream, 
    @strErrorMessage='writing to the file "'+@FileAndPath+'"'
if @HR=0 execute @hr = sp_OAMethod  @objTextStream, 'Write', Null, @String

if @HR=0 Select @objErrorObject=@objTextStream, @strErrorMessage='closing the file "'+@FileAndPath+'"'
if @HR=0 execute @hr = sp_OAMethod  @objTextStream, 'Close'

if @hr<>0
    begin
    Declare 
        @Source varchar(255),
        @Description Varchar(255),
        @Helpfile Varchar(255),
        @HelpID int

    EXECUTE sp_OAGetErrorInfo  @objErrorObject, 
        @source output,@Description output,@Helpfile output,@HelpID output
    Select @strErrorMessage='Error whilst '
            +coalesce(@strErrorMessage,'doing something')
            +', '+coalesce(@Description,'')
    raiserror (@strErrorMessage,16,1)
    end
EXECUTE  sp_OADestroy @objTextStream
EXECUTE sp_OADestroy @objTextStream